In [1]:
import pandas as pd
import numpy as np

In [2]:
df= pd.read_csv('/Users/morgan/Desktop/SEIS 763 ML/posting_32556_salary_info& company info.csv',header=0)

In [3]:
df.head()

,job_id,company_name,title,description,max_salary,location,company_id,views,min_salary,formatted_work_type,...,headquarter state,country,headquarter city,headquarter zipcode,headquarter addr,industry,speciality,employee_count,follower_count,time_recorded
0,921716,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,41600.0,"Princeton, NJ",2774458.0,20.0,35360.0,Full-time,...,NJ,US,Jersey City,07302,242 Tenth Street,Real Estate,"real estate, new development",402,2351,1970-01-20 19:56:49.275
1,10998357,The National Exemplar,Assitant Restaurant Manager,The National Exemplar is accepting application...,65000.0,"Cincinnati, OH",64896719.0,8.0,45000.0,Full-time,...,Ohio,US,Mariemont,45227,6880 Wooster Pike,Restaurants,NaN,15,40,1970-01-20 19:54:39.321
2,23221523,"Abrams Fensterman, LLP",Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...,175000.0,"New Hyde Park, NY",766262.0,16.0,140000.0,Full-time,...,New York,US,Lake Success,11042,3 Dakota Drive,Law Practice,"Civil Litigation, Corporate & Securities Law, ...",222,2427,1970-01-20 19:48:17.299
3,103254301,Raw Cereal,Producer,Company DescriptionRaw Cereal is a creative de...,300000.0,United States,81942316.0,7.0,60000.0,Contract,...,CA,US,Los Angeles,90017,0,Design Services,NaN,11,447,1970-01-20 19:47:41.202
4,95428182,CLEVELAND KIDS BOOK BANK,Administrative Coordinator,Job Title: Administrative CoordinatorOrganizat...,NaN,"Cleveland, OH",55624331.0,1.0,NaN,Full-time,...,Ohio,US,Cleveland,44114,3635 PERKINS AVE,Non-profit Organizations,NaN,6,289,1970-01-20 19:47:36.403


In [4]:
df.columns

Index(['job_id', 'company_name', 'title', 'description', 'max_salary',
       'location', 'company_id', 'views', 'min_salary', 'formatted_work_type',
       'original_listed_time', 'expiry', 'formatted_experience_level',
       'listed_time', 'work_type', 'currency', 'normalized_salary', 'zip_code',
       'name', 'company desc', 'company_size', 'headquarter state', 'country',
       'headquarter city', 'headquarter zipcode', 'headquarter addr',
       'industry', 'speciality', 'employee_count', 'follower_count',
       'time_recorded'],
      dtype='object')

In [5]:
import nltk

In [6]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [7]:
# Download NLTK data
#nltk.download('punkt')
#nltk.download('stopwords')

In [8]:
# Define keyword lists for extraction and classification
degree_keywords =  {
    "bachelor": ["bachelor", "undergraduate","4 year"],
    "postgraduate": ["master", "postgraduate", "phd", "doctorate","mba"],
}

major_keywords = ["data science", "artificial intelligence", "MIS", "analytics", "business analysis","information system"]

data_keywords = ["data"]

hard_skills = {
    "programming_languages": ["python", "java", "c++", "r", "sql", "javascript","matlab"],
    "analysis_statistics": ["excel", "tableau", "statistics", "sas", "spss","regression","decision tree","predictive analysis",
                            "statistics","statistical",'models'],
    "machine_learning": ["tensorflow", "keras", "scikit-learn", "pytorch", "deep learning","nlp","svm",
                         "clustering","natural language","machine learning", "deeep learning","ai"],
    "data_visualization": ["power bi", "matplotlib", "seaborn", "dash","tableau","cognos",'bi'],
    "cloud_platform": ["aws", "azure", "google cloud","crm","cloud computing"],
    "database": ["dataset","mysql", "postgresql", "mongodb", "oracle","sql","database","query"],
    "big_data": ["hadoop", "spark", "big data", "kafka","bigquery"]
}

soft_skills = ["communication", "teamwork", "problem-solving", "adaptability", "leadership", "creativity","detail","deadline","collaboration"]

"""business_keywords = [
    "business analysis", "market", "strategy", "management", "finance",
    "operation", "sales", "marketing", "client", "consulting", "stakeholder",
    "crm", "leadership", "planning", "kpi","agile","project management","market research",
    "interlligence","business analysis", "mba", "market", "excel","dax","jira","erp","customer", "google analysis","visio",
    "azure"
]

engineering_keywords = [
    "developer", "engineer", "scientist","software", "programming", "ai", "ml", "database",
    "backend", "frontend", "data structures", "cloud", "API", "integration",
    "ETL", "devops", "algorithm", "distributed systems", "coding", "developer"
]"""

'business_keywords = [\n    "business analysis", "market", "strategy", "management", "finance",\n    "operation", "sales", "marketing", "client", "consulting", "stakeholder",\n    "crm", "leadership", "planning", "kpi","agile","project management","market research",\n    "interlligence","business analysis", "mba", "market", "excel","dax","jira","erp","customer", "google analysis","visio",\n    "azure"\n]\n\nengineering_keywords = [\n    "developer", "engineer", "scientist","software", "programming", "ai", "ml", "database",\n    "backend", "frontend", "data structures", "cloud", "API", "integration",\n    "ETL", "devops", "algorithm", "distributed systems", "coding", "developer"\n]'

In [9]:
# Add scoring columns
def initialize_columns(df):
    # Ensure all extracted feature columns are initialized with appropriate data types
    df['degree_level'] = ""  # Default is an empty string for degree levels
    df['is_data_related'] = False  # Initialize as boolean
    df['job_classification'] = ""  # Default is an empty string
    df['soft_skills'] = ""  # Default is an empty string

    # Initialize score columns with integers (default 0)
    df['programming_languages_score'] = 0
    df['analysis_statistics_score'] = 0
    df['machine_learning_score'] = 0
    df['data_visualization_score'] = 0
    df['cloud_platform_score'] = 0
    df['database_score'] = 0
    df['big_data_tools_score'] = 0
    
    hard_skills_categories = list(hard_skills.keys())
    for category in hard_skills_categories:
        df[f"{category}_score"] = 0
    df['business_engineering_score'] = 0
    return df

In [64]:
def classify_job(title, description):
    """
    Classifies a job into 'Business' or 'Engineering' based on keywords.

    Args:
        title: The job title.
        description: The job description.

    Returns:
        'Business' or 'Engineering'
    """

    description = str(description).lower()
    title = str(title).lower()
    
    engineering_keywords = [
        "developer", "engineer", "software", "programming", "ai", "ml", "database",
        "backend", "frontend", "cloud", "api", "integration", "etl", "devops", "algorithm",
        "distributed systems", "coding", "data mining", "data structures"
    ]
    
    business_keywords = [
        "business strategy", "market research", "kpi", "stakeholder management", 
        "client engagement", "financial modeling", "sales pipeline", 
        "consulting", "mba", "leadership", "planning", "marketing campaign","business analysis",
        "business interlligence"
    ]

    
    # Combine keywords and job title for comprehensive analysis
    text = f"{str(title).lower()} {str(description).lower()}"
    
    engineering_score = sum(keyword in text for keyword in engineering_keywords)
    business_score = sum(keyword in text for keyword in business_keywords)
    
    if engineering_score > business_score:
        return "Engineering"
    elif business_score > engineering_score:
        return "Business"
    else:
        return "Other"

In [11]:
#df['Job Category'] = df.apply(lambda x: classify_job(x['title'], x['description']), axis=1)

In [12]:
#df[df['Job Category']=="Engineering"]

In [13]:
# Function to extract and score information
def extract_info_with_scores(description):
    degree = []
    data_related = False
    category_scores = {category: 0 for category in hard_skills.keys()}
    extracted_soft_skills = []
    
    if isinstance(description, str):
        description_lower = description.lower()
        
        # Degree keywords
        for word in degree_keywords:
            if word in description_lower:
                degree.append(word)
        
        # Data-related keywords
        for word in data_keywords:
            if word in description_lower:
                data_related = True

        # Hard skills scoring
        for category, keywords in hard_skills.items():
            for keyword in keywords:
                if keyword in description_lower:
                    category_scores[category] += 1
        
        # Soft skills extraction
        for skill in soft_skills:
            if skill in description_lower:
                extracted_soft_skills.append(skill)
    
    return degree, data_related, category_scores, extracted_soft_skills

In [38]:
# Apply processing
def process_job_postings(df):
    # Initialize columns
    df = initialize_columns(df)
    
    # Loop through rows
    for idx, row in df.iterrows():
        description = row['description']
        
        # Extract information and scores
        degree, data_related, category_scores, soft_skills_extracted = extract_info_with_scores(description)
        job_classification = classify_job(row['title'], row['description'])
        
        # Update columns
        df.at[idx, 'degree_level'] = ', '.join(degree)
        df.at[idx, 'is_data_related'] = data_related
        df.at[idx, 'soft_skills'] = ', '.join(soft_skills_extracted)
        df.at[idx, 'job_classification'] = job_classification
        
        # Update hard skill scores
        for category, score in category_scores.items():
            df.at[idx, f"{category}_score"] = score
    
    return df

In [66]:
df = process_job_postings(df)

In [26]:
df.shape

(32555, 44)

In [68]:
business_jobs=df[(df['job_classification']== "Business")&(df['is_data_related']==True)]
business_jobs

,job_id,company_name,title,description,max_salary,location,company_id,views,min_salary,formatted_work_type,...,soft_skills,programming_languages_score,analysis_statistics_score,machine_learning_score,data_visualization_score,cloud_platform_score,database_score,big_data_tools_score,big_data_score,business_engineering_score
44,3514683108,AVAVA,Associate Brand Manager,Job Title: Associate Brand Manager / Brand Man...,90000.0,"Waltham, MA",40692980.0,1.0,75000.0,Full-time,...,"communication, problem-solving, leadership",1,1,1,1,1,0,0,0,0
195,3862107843,Swipeclock,Product Manager,Swipeclock is on a mission to make working lif...,120000.0,"South Jordan, UT",940888.0,3.0,100000.0,Full-time,...,"communication, problem-solving, detail",1,1,1,1,0,0,0,0,0
215,3870701883,EmazingGroup - INTO THE AM & iHeartRaves,Merchandise Planner,"Top Reasons to Work with Us:\nSince 2010, the ...",75000.0,"Anaheim, CA",3696072.0,7.0,65000.0,Full-time,...,"adaptability, detail, deadline",1,2,1,1,1,0,0,0,0
217,3871144579,World of Reza,Head of Client Relations,"\nDo you have a passion for beautiful, timeles...",110000.0,New York City Metropolitan Area,78331809.0,6.0,85000.0,Full-time,...,"communication, leadership, detail",1,1,1,1,1,0,0,0,0
333,3884432698,Dahl Consulting,Full Manager Bookkeeper (26394),Seeking a remote job opportunity? Check out th...,75000.0,United States,29686.0,26.0,65000.0,Full-time,...,"problem-solving, detail",1,0,1,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32393,3906257446,KeyBank,Financial Wellness Associate,Location:\n\n405 NW Gilman Boulevard - Issaqua...,45760.0,"Issaquah, WA",3252.0,3.0,39520.0,Full-time,...,"communication, teamwork, adaptability, leaders...",1,1,1,1,0,0,0,0,0
32400,3906257580,Advantage Solutions,"Director, Client Services","Director Client Services \n\nAt our Company, w...",105000.0,"Bentonville, AR",9123.0,4.0,75000.0,Full-time,...,"communication, leadership, detail, deadline",1,1,1,1,0,0,0,0,0
32411,3906258100,TD,Senior Finance Analyst (US),Work Location:\n\n1006 Astoria Boulevard\n\nHo...,112320.0,"Astoria, OR",2775.0,5.0,68640.0,Full-time,...,"communication, teamwork, leadership, creativit...",1,0,1,2,0,0,0,0,0
32438,3906258582,Veolia North America,Administrative Coordinator,Company Description\n\nAbout Veolia North Amer...,70000.0,"West Nyack, NY",508963.0,2.0,60000.0,Full-time,...,"communication, deadline",1,1,1,1,0,0,0,0,0


In [70]:
business_jobs.describe()

,job_id,max_salary,company_id,views,min_salary,normalized_salary,zip_code,company_size,employee_count,follower_count,programming_languages_score,analysis_statistics_score,machine_learning_score,data_visualization_score,cloud_platform_score,database_score,big_data_tools_score,big_data_score,business_engineering_score
count,9.800000e+02,905.000000,9.800000e+02,974.000000,905.000000,980.000000,761.000000,944.000000,980.000000,9.800000e+02,980.000000,980.000000,980.000000,980.000000,980.000000,980.000000,980.0,980.000000,980.0
mean,3.899246e+09,146087.050751,1.012180e+07,47.102669,105470.384232,123119.779122,52968.714849,5.010593,26781.190816,9.789816e+05,1.059184,1.088776,0.969388,1.148980,0.282653,0.111224,0.0,0.009184,0.0
std,1.406554e+07,72586.378800,2.351304e+07,392.957084,52175.203725,61995.642655,33349.200463,2.084100,75891.326422,3.267192e+06,0.297362,0.842728,0.249784,0.484962,0.470482,0.359997,0.0,0.095439,0.0
min,3.514683e+09,35.000000,1.016000e+03,1.000000,30.000000,32.500000,1201.000000,1.000000,1.000000,1.400000e+01,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0
25%,3.895461e+09,95000.000000,5.304750e+03,4.000000,71289.000000,80000.000000,20190.000000,3.000000,277.750000,1.854000e+04,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.0
50%,3.902339e+09,135000.000000,1.641060e+05,5.000000,94900.000000,111300.000000,57201.000000,5.000000,2201.000000,1.100875e+05,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.0,0.000000,0.0
75%,3.904943e+09,184000.000000,3.706728e+06,15.000000,132300.000000,155000.000000,90001.000000,7.000000,22839.250000,4.931160e+05,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.0,0.000000,0.0
max,3.906261e+09,520000.000000,1.026498e+08,9975.000000,416000.000000,500000.000000,99501.000000,7.000000,748312.000000,3.270284e+07,4.000000,6.000000,3.000000,4.000000,3.000000,4.000000,0.0,1.000000,0.0


In [72]:
engineering_jobs=df[(df['job_classification']== "Engineering")&(df['is_data_related']==True)]
engineering_jobs

,job_id,company_name,title,description,max_salary,location,company_id,views,min_salary,formatted_work_type,...,soft_skills,programming_languages_score,analysis_statistics_score,machine_learning_score,data_visualization_score,cloud_platform_score,database_score,big_data_tools_score,big_data_score,business_engineering_score
4,95428182,CLEVELAND KIDS BOOK BANK,Administrative Coordinator,Job Title: Administrative CoordinatorOrganizat...,NaN,"Cleveland, OH",55624331.0,1.0,NaN,Full-time,...,"communication, creativity, detail, collaboration",1,1,1,1,2,1,0,0,0
11,1014822088,Tracker,Marketing Specialist,About TrackerTracker is a leading global cloud...,80000.0,United States,255252.0,7.0,70000.0,Full-time,...,"detail, deadline",1,0,1,1,1,0,0,0,0
13,1448163866,Genlabs Corp,Office Manager,Responsibilities: • Oversees and manages all a...,70000.0,"Chino, CA",4273342.0,3.0,65000.0,Full-time,...,"communication, detail",1,1,1,1,0,0,0,0,0
18,2558399667,"I.T. Solutions, Inc.","Validation Engineer, Labware LIMS","Validation Engineer, Labware LIMSFoster City, ...",145600.0,"Foster City, CA",2610793.0,NaN,124800.0,Contract,...,communication,1,0,0,1,0,1,0,0,0
22,2920450495,Grunwald Mechanical Contractors & Engineers,Service Coordinator,"QualificationsExperience:Data Entry, 4 years (...",75000.0,"Omaha, NE",80316732.0,4.0,55000.0,Full-time,...,"communication, problem-solving, detail",1,1,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32537,3906262655,Vertisystem,Human Resources Coordinator,JOB DESCRIPTION:Job Title:HR Recruiter - Profe...,72800.0,"Greer, SC",3009879.0,2.0,62400.0,Contract,...,communication,1,0,1,1,1,0,0,0,0
32539,3906262691,"TalentBurst, an Inc 5000 company",Senior Validation Engineer,"Title: Validation Engineer III, Req#: ROCGJP00...",156000.0,"Oceanside, CA",122451.0,1.0,104000.0,Contract,...,"communication, teamwork, problem-solving, detail",1,2,1,1,0,0,0,0,0
32540,3906263026,Applicantz,Desktop Test Engineer,VISA SPONSORSHIP NOT AVAILABLE. Work to be don...,71718.4,United States,24629.0,3.0,71718.4,Contract,...,"communication, creativity, detail, deadline",1,2,1,1,0,0,0,0,0
32546,3906265006,Applicantz,SAP FICO Functional Consultant,VISA SPONSORSHIP IS NOT AVAILABLE. Remote role...,186492.8,United States,24629.0,3.0,186492.8,Contract,...,"creativity, detail",1,1,1,1,0,0,0,0,0


In [74]:
data_job=df[(df['is_data_related']==True) & (df['degree_level']!='')& (df['job_classification']!= "Unknown")]

In [88]:
data_job2=df[(df['is_data_related']==True) & (df['job_classification']!= "Unknown")]

In [90]:
data_job2.shape

(12684, 44)

In [94]:
data_job2[(data_job2['job_classification']== "Business")]

,job_id,company_name,title,description,max_salary,location,company_id,views,min_salary,formatted_work_type,...,soft_skills,programming_languages_score,analysis_statistics_score,machine_learning_score,data_visualization_score,cloud_platform_score,database_score,big_data_tools_score,big_data_score,business_engineering_score
44,3514683108,AVAVA,Associate Brand Manager,Job Title: Associate Brand Manager / Brand Man...,90000.0,"Waltham, MA",40692980.0,1.0,75000.0,Full-time,...,"communication, problem-solving, leadership",1,1,1,1,1,0,0,0,0
195,3862107843,Swipeclock,Product Manager,Swipeclock is on a mission to make working lif...,120000.0,"South Jordan, UT",940888.0,3.0,100000.0,Full-time,...,"communication, problem-solving, detail",1,1,1,1,0,0,0,0,0
215,3870701883,EmazingGroup - INTO THE AM & iHeartRaves,Merchandise Planner,"Top Reasons to Work with Us:\nSince 2010, the ...",75000.0,"Anaheim, CA",3696072.0,7.0,65000.0,Full-time,...,"adaptability, detail, deadline",1,2,1,1,1,0,0,0,0
217,3871144579,World of Reza,Head of Client Relations,"\nDo you have a passion for beautiful, timeles...",110000.0,New York City Metropolitan Area,78331809.0,6.0,85000.0,Full-time,...,"communication, leadership, detail",1,1,1,1,1,0,0,0,0
333,3884432698,Dahl Consulting,Full Manager Bookkeeper (26394),Seeking a remote job opportunity? Check out th...,75000.0,United States,29686.0,26.0,65000.0,Full-time,...,"problem-solving, detail",1,0,1,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32393,3906257446,KeyBank,Financial Wellness Associate,Location:\n\n405 NW Gilman Boulevard - Issaqua...,45760.0,"Issaquah, WA",3252.0,3.0,39520.0,Full-time,...,"communication, teamwork, adaptability, leaders...",1,1,1,1,0,0,0,0,0
32400,3906257580,Advantage Solutions,"Director, Client Services","Director Client Services \n\nAt our Company, w...",105000.0,"Bentonville, AR",9123.0,4.0,75000.0,Full-time,...,"communication, leadership, detail, deadline",1,1,1,1,0,0,0,0,0
32411,3906258100,TD,Senior Finance Analyst (US),Work Location:\n\n1006 Astoria Boulevard\n\nHo...,112320.0,"Astoria, OR",2775.0,5.0,68640.0,Full-time,...,"communication, teamwork, leadership, creativit...",1,0,1,2,0,0,0,0,0
32438,3906258582,Veolia North America,Administrative Coordinator,Company Description\n\nAbout Veolia North Amer...,70000.0,"West Nyack, NY",508963.0,2.0,60000.0,Full-time,...,"communication, deadline",1,1,1,1,0,0,0,0,0


In [100]:
data_job2[(data_job2['job_classification']== "Engineering")]

,job_id,company_name,title,description,max_salary,location,company_id,views,min_salary,formatted_work_type,...,soft_skills,programming_languages_score,analysis_statistics_score,machine_learning_score,data_visualization_score,cloud_platform_score,database_score,big_data_tools_score,big_data_score,business_engineering_score
4,95428182,CLEVELAND KIDS BOOK BANK,Administrative Coordinator,Job Title: Administrative CoordinatorOrganizat...,NaN,"Cleveland, OH",55624331.0,1.0,NaN,Full-time,...,"communication, creativity, detail, collaboration",1,1,1,1,2,1,0,0,0
11,1014822088,Tracker,Marketing Specialist,About TrackerTracker is a leading global cloud...,80000.0,United States,255252.0,7.0,70000.0,Full-time,...,"detail, deadline",1,0,1,1,1,0,0,0,0
13,1448163866,Genlabs Corp,Office Manager,Responsibilities: • Oversees and manages all a...,70000.0,"Chino, CA",4273342.0,3.0,65000.0,Full-time,...,"communication, detail",1,1,1,1,0,0,0,0,0
18,2558399667,"I.T. Solutions, Inc.","Validation Engineer, Labware LIMS","Validation Engineer, Labware LIMSFoster City, ...",145600.0,"Foster City, CA",2610793.0,NaN,124800.0,Contract,...,communication,1,0,0,1,0,1,0,0,0
22,2920450495,Grunwald Mechanical Contractors & Engineers,Service Coordinator,"QualificationsExperience:Data Entry, 4 years (...",75000.0,"Omaha, NE",80316732.0,4.0,55000.0,Full-time,...,"communication, problem-solving, detail",1,1,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32537,3906262655,Vertisystem,Human Resources Coordinator,JOB DESCRIPTION:Job Title:HR Recruiter - Profe...,72800.0,"Greer, SC",3009879.0,2.0,62400.0,Contract,...,communication,1,0,1,1,1,0,0,0,0
32539,3906262691,"TalentBurst, an Inc 5000 company",Senior Validation Engineer,"Title: Validation Engineer III, Req#: ROCGJP00...",156000.0,"Oceanside, CA",122451.0,1.0,104000.0,Contract,...,"communication, teamwork, problem-solving, detail",1,2,1,1,0,0,0,0,0
32540,3906263026,Applicantz,Desktop Test Engineer,VISA SPONSORSHIP NOT AVAILABLE. Work to be don...,71718.4,United States,24629.0,3.0,71718.4,Contract,...,"communication, creativity, detail, deadline",1,2,1,1,0,0,0,0,0
32546,3906265006,Applicantz,SAP FICO Functional Consultant,VISA SPONSORSHIP IS NOT AVAILABLE. Remote role...,186492.8,United States,24629.0,3.0,186492.8,Contract,...,"creativity, detail",1,1,1,1,0,0,0,0,0


In [102]:
data_job[(data_job['job_classification']== "Business")]

,job_id,company_name,title,description,max_salary,location,company_id,views,min_salary,formatted_work_type,...,soft_skills,programming_languages_score,analysis_statistics_score,machine_learning_score,data_visualization_score,cloud_platform_score,database_score,big_data_tools_score,big_data_score,business_engineering_score
44,3514683108,AVAVA,Associate Brand Manager,Job Title: Associate Brand Manager / Brand Man...,90000.0,"Waltham, MA",40692980.0,1.0,75000.0,Full-time,...,"communication, problem-solving, leadership",1,1,1,1,1,0,0,0,0
195,3862107843,Swipeclock,Product Manager,Swipeclock is on a mission to make working lif...,120000.0,"South Jordan, UT",940888.0,3.0,100000.0,Full-time,...,"communication, problem-solving, detail",1,1,1,1,0,0,0,0,0
441,3884435818,MiQ,"VP Agency Partnerships, Indies","Role: VP Agency Partnerships, Indies\n\nLocati...",250000.0,"New York, NY",2883353.0,38.0,210000.0,Full-time,...,"communication, problem-solving, leadership, de...",1,1,1,1,0,0,0,0,0
550,3884438827,Exact Sciences,Experience Product Manager Associate Director,"Help us change lives\n\nAt Exact Sciences, we’...",242000.0,"Madison, WI",48835.0,38.0,152000.0,Full-time,...,"communication, teamwork, problem-solving, lead...",1,2,1,2,0,0,0,0,0
551,3884438893,Planet Pharma,Associate Director Market Research Competitive...,Job Description\n\nDuties\n\nThe Senior Associ...,187200.0,"Fairfield County, CT",2474662.0,10.0,124800.0,Full-time,...,"communication, leadership",1,1,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32143,3906252922,Wolters Kluwer,Pricing Manager - Remote,"At Wolters Kluwer, our mission is to deliver d...",164700.0,"Texas, United States",2483.0,4.0,117500.0,Full-time,...,"communication, leadership, collaboration",1,2,1,1,0,1,0,0,0
32157,3906253168,Visa,Sr Director - Consulting & Analytics - Nationa...,Company Description\n\nVisa is a world leader ...,301150.0,"Washington, DC",2190.0,4.0,173400.0,Full-time,...,"communication, leadership, detail",1,1,1,1,0,0,0,0,0
32159,3906253175,Visa,"Program Manager, Client Consulting, VCA Risk P...",Company Description\n\nVisa is a world leader ...,191800.0,"Ashburn, VA",2190.0,4.0,112500.0,Full-time,...,"communication, leadership, detail",1,1,1,1,0,0,0,0,0
32160,3906253176,Visa,"Program Manager, Client Consulting, VCA Risk P...",Company Description\n\nVisa is a world leader ...,191800.0,"Foster City, CA",2190.0,4.0,112500.0,Full-time,...,"communication, leadership, detail",1,1,1,1,0,0,0,0,0


In [104]:
data_job[(data_job['job_classification']== "Engineering")]

,job_id,company_name,title,description,max_salary,location,company_id,views,min_salary,formatted_work_type,...,soft_skills,programming_languages_score,analysis_statistics_score,machine_learning_score,data_visualization_score,cloud_platform_score,database_score,big_data_tools_score,big_data_score,business_engineering_score
4,95428182,CLEVELAND KIDS BOOK BANK,Administrative Coordinator,Job Title: Administrative CoordinatorOrganizat...,NaN,"Cleveland, OH",55624331.0,1.0,NaN,Full-time,...,"communication, creativity, detail, collaboration",1,1,1,1,2,1,0,0,0
11,1014822088,Tracker,Marketing Specialist,About TrackerTracker is a leading global cloud...,80000.0,United States,255252.0,7.0,70000.0,Full-time,...,"detail, deadline",1,0,1,1,1,0,0,0,0
30,3189117072,"Signature Estate & Investment Advisors, LLC",Client Service Associate / Practice Manager,Company OverviewSignature Estate and Investmen...,100000.0,"Redondo Beach, CA",1299114.0,6.0,70000.0,Full-time,...,"communication, detail, deadline",1,0,1,1,1,1,0,0,0
32,3260455681,Weyerhaeuser,Operations Specialist,Weyerhaeuser is searching for a Operations Spe...,109113.0,"Castle Rock, WA",4494.0,18.0,72742.0,Full-time,...,"communication, leadership, detail",1,1,1,1,1,0,0,0,0
35,3297645846,"Navajo Transitional Energy Company, LLC",Contract Administrator,Contract AdministratorSummary of PositionThe C...,90000.0,"Broomfield, CO",59475546.0,6.0,74400.0,Full-time,...,"communication, detail",1,1,1,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32501,3906260068,Workday,Senior Associate Quality Assurance Engineer,Your work days are brighter here.\n\nAt Workda...,151200.0,"Atlanta, GA",17719.0,4.0,74500.0,Full-time,...,"communication, detail",3,2,1,1,1,0,0,0,0
32519,3906260689,Volkswagen Group of America Innovation and Eng...,iOS Developer,VW. Two letters. Endless opportunities. A brig...,190000.0,"Belmont, CA",29308364.0,5.0,128400.0,Full-time,...,communication,4,0,1,1,0,0,0,0,0
32532,3906262009,Solugenix,Senior Business Intelligence Developer,Senior Business Intelligence DeveloperLos Ange...,141440.0,"Los Angeles, CA",43325.0,3.0,133120.0,Contract,...,"communication, teamwork, problem-solving, detail",2,3,1,3,0,2,0,0,0
32533,3906262044,Legal.io,Privacy Program Manager,Legal.io is working with a leading automotive ...,187200.0,United States,6452409.0,3.0,135200.0,Contract,...,"communication, detail",1,0,1,1,0,0,0,0,0


In [132]:
# Filter the DataFrame for rows where normalized_salary is in the specified range
filtered_data = data_job[(data_job["normalized_salary"] > 5000) & (data_job["normalized_salary"] < 20000)]

# Update the normalized_salary column by multiplying it by 12
data_job.loc[(data_job["normalized_salary"] > 5000) & (data_job["normalized_salary"] < 20000), "normalized_salary"] *= 12

In [156]:
# Filter the DataFrame for rows where normalized_salary is in the specified range
filtered_data1 = data_job[(data_job["normalized_salary"] < 1000)]

data_job.loc[data_job["normalized_salary"] < 1000, "normalized_salary"] =((data_job["max_salary"] + data_job["min_salary"]) / 2 * 2080)


In [158]:
data_job["normalized_salary"].describe()

count    5.958000e+03
mean     1.313777e+05
std      1.327843e+06
min      2.000050e+04
25%      7.550000e+04
50%      1.050000e+05
75%      1.400000e+05
max      1.025000e+08
Name: normalized_salary, dtype: float64

In [ ]:
# Save the processed dataset
#data_job.to_csv('processed_job_postings_nltk_v2.csv', index=False)